In [1]:
"""
References: 
1) https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-from-scratch-for-mnist-handwritten-digit-classification/
2) https://machinelearningmastery.com/visualize-deep-learning-neural-network-model-keras/
3) King's College London Keras Instruction Tutorial PDF
4) https://medium.com/mini-distill/effect-of-batch-size-on-training-dynamics-21c14f7a716e
""" 

"\nReferences: \n1) https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-from-scratch-for-mnist-handwritten-digit-classification/\n2) https://machinelearningmastery.com/visualize-deep-learning-neural-network-model-keras/\n3) King's College London Keras Instruction Tutorial PDF\n4) https://medium.com/mini-distill/effect-of-batch-size-on-training-dynamics-21c14f7a716e\n"

In [2]:
#change width of Jupyer Notebook to use the whole window
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))     

C:\Users\cipri\AppData\Local\Temp\ipykernel_28512\4283796252.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import SGD

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import numpy as np
from numpy import mean
from numpy import std

from matplotlib import pyplot as plt

import statistics

In [4]:
# import libraries to check available training devices
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras import mixed_precision

# list available devices
print(device_lib.list_local_devices())

# GPU performance optimisations - only works with XLA devices on non-windows machines
tf.function(jit_compile=True)                                   

# makes use of the tensor cores available on RTX cards
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# print compute and variable precision 
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11609564209158597473
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22385000448
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17626220348752568767
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:31:00.0, compute capability: 8.9"
xla_global_id: 416903419
]
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4090, compute capability 8.9
Compute dtype: float16
Variable dtype: float32


In [5]:
# load dataset and perform split
def load_dataset():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()                   # load dataset
    
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))                   # reshape X_train to have a single channel 28x28 img representation
    X_test  = X_test.reshape((X_test.shape[0], 28, 28, 1))                    # reshape X_test to have a single channel 28x28 img representation
    
    y_train = to_categorical(y_train)                                          # one hot encode train labels
    y_test  = to_categorical(y_test)                                           # one hot encode test labels
        
    return X_train, y_train, X_test, y_test

# process dataset by changing 255 integers to float 0.0-1.0 values
def prep_pixels(train, test):
    train_float = train.astype('float32') / 255.0                              # normalize to range 0-1
    test_float  = test.astype('float32') / 255.0                               # normalize to range 0-1
     
    return train_float, test_float                                             # return normalized images

In [6]:
# define CNN network
def define_cnn():
    CNN = Sequential()
    CNN.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
    CNN.add(BatchNormalization())
    CNN.add(Conv2D(32,kernel_size=3,activation='relu'))
    CNN.add(BatchNormalization())
    CNN.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
    CNN.add(BatchNormalization())
    CNN.add(Dropout(0.4))
    
    CNN.add(Conv2D(64,kernel_size=3,activation='relu'))
    CNN.add(BatchNormalization())
    CNN.add(Conv2D(64,kernel_size=3,activation='relu'))
    CNN.add(BatchNormalization())
    CNN.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
    CNN.add(BatchNormalization())
    CNN.add(Dropout(0.4))
    
    CNN.add(Flatten())
    CNN.add(Dense(128, activation='relu'))
    CNN.add(BatchNormalization())
    CNN.add(Dropout(0.4))
    CNN.add(Dense(10, activation='softmax'))
    
    
    # compile model
    #optimizer = SGD(learning_rate=0.01, momentum=0.9)
    CNN.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    
    # return defined cnn
    return CNN

# train CNN network
def train_model(X_train, y_train, X_test, y_test, n_epochs, b_size):
    # fit model
    cnn_model = define_cnn()                                      
    history = cnn_model.fit(X_train, y_train, 
                            epochs = n_epochs, 
                            batch_size = b_size, 
                            validation_data=(X_test, y_test), 
                            verbose=1)
    
    return cnn_model, history
    

In [7]:
# evaluate a model using k-fold cross-validation
def evaluate_model(X_data, y_data, n_folds, n_epochs, b_size):
    scores, histories = list(), list()
    kfold = KFold(n_folds, shuffle = True, random_state = 1)                        # prepare cross validation

    for train_ix, test_ix in kfold.split(X_data):                                 # enumerate splits
        X_train, y_train = X_data[train_ix], y_data[train_ix]                    # select rows for training
        X_test, y_test   = X_data[test_ix], y_data[test_ix]                      # select rows for testing
        
        cnn_model, history = train_model(X_train, y_train, X_test, y_test, n_folds, n_epochs)

        # stores scores
        _, acc = cnn_model.evaluate(testX, testY, verbose=0)
        print('> %.3f' % (acc * 100.0))
        
        scores.append(accuracy)
        histories.append(history)
        
    return scores, histories                                                    # return evaluation   
    
# performance metrics
def performance(X_test, y_test, model):
        # evaluate model
        yhat_probs = model.predict(X_test, verbose=0)                           # predict probabilities for test set
        yhat_classes = np.argmax(yhat_probs, axis=1)                             # predict crisp classes for test set 
        y_test = np.argmax(y_test, axis=1)
        
        #print(y_test)
        #print(yhat_classes)
        
        # accuracy: (tp + tn) / (p + n)
        accuracy = accuracy_score(y_test, yhat_classes)*100
        print('Accuracy: %f' % accuracy)
        
        # precision tp / (tp + fp)
        precision = precision_score(y_test, yhat_classes, average = None)*100
        print("Precision: ", statistics.fmean(precision))
        print(", ".join('{:0.2f}'.format(i) for i in precision))
        
        # recall: tp / (tp + fn)
        recall = recall_score(y_test, yhat_classes, average = None)*100
        print("Recall: ", statistics.fmean(recall))
        print(", ".join('{:0.2f}'.format(i) for i in recall))
        
        # f1: 2 tp / (2 tp + fp + fn)
        f1 = f1_score(y_test, yhat_classes, average = None)*100
        print("F1 Score: ", statistics.fmean(f1))
        print(", ".join('{:0.2f}'.format(i) for i in f1))
        
        # kappa
        kappa = cohen_kappa_score(y_test, yhat_classes)*100
        print('Cohens kappa: %f' % kappa)
        
        # confusion matrix
        cm = confusion_matrix(y_test, yhat_classes)
        labels = ("0", "1", "2", "3", "4", "5", "6", "7", "8", "9")
        cm_plot = ConfusionMatrixDisplay(confusion_matrix=cm, 
                                         display_labels = labels)
        cm_plot.plot()
        plt.show()

# plot diagnostic learning curves
def summarize_diagnostics(history):
    if type(history) == list:
        for i in range(len(history)):
            # plot loss
            plt.subplot(2, 1, 1)
            plt.title('Cross Entropy Loss')
            plt.plot(history[i].history['loss'], color='blue', label='train')
            plt.plot(history[i].history['val_loss'], color='orange', label='test')

            # plot accuracy
            plt.subplot(2, 1, 2)
            plt.title('Classification Accuracy')
            plt.plot(history[i].history['accuracy'], color='blue', label='train')
            plt.plot(history[i].history['val_accuracy'], color='orange', label='test')
            plt.show()
            
    else:
        # plot loss
        plt.subplot(2, 1, 1)
        plt.title('Cross Entropy Loss')
        plt.plot(history.history['loss'], color='blue', label='train')
        plt.plot(history.history['val_loss'], color='orange', label='test')

        # plot accuracy
        plt.subplot(2, 1, 2)
        plt.title('Classification Accuracy')
        plt.plot(history.history['accuracy'], color='blue', label='train')
        plt.plot(history.history['val_accuracy'], color='orange', label='test')
        plt.show()
        
# display performance summary
def summarize_performance(scores):
    print('Accuracy: mean=%.3f std=%.3f, n=%d' % 
          (mean(scores)*100, std(scores)*100, len(scores))) 
    
    plt.boxplot(scores)                                                       # box and whisker plots of results
    plt.show()                                                                # display results

def main_code():
    with tf.device('/device:GPU:0'):
        EPOCHS  = 40
        BATCHES = 16

        X_train, y_train, X_test, y_test = load_dataset()                            # load dataset
        X_train, X_test = prep_pixels(X_train, X_test)                               # prepare pixel data
        #scores, history = evaluate_model(X_train, y_train, 5, EPOCHS, BATCHES)     # evaluate model
        #summarize_performance(scores)                                                # summarize estimated performance
        cnn_model, history = train_model(X_train, y_train, X_test, y_test, EPOCHS, BATCHES)
        performance(X_test, y_test, cnn_model)
        summarize_diagnostics(history)                                             # display learning curves
        
        # release allocated resources after termination
        tf.keras.backend.clear_session()
        #cuda.select_device(0)
        #cuda.close()

In [ ]:
# guard boilerplate
if __name__ == "__main__":  
    main_code()

Epoch 1/40
3750/3750 [==============================] - 118s 30ms/step - loss: 0.2019 - accuracy: 0.9397 - val_loss: 0.0428 - val_accuracy: 0.9852
Epoch 2/40
3750/3750 [==============================] - 112s 30ms/step - loss: 0.0852 - accuracy: 0.9741 - val_loss: 0.0257 - val_accuracy: 0.9911
Epoch 3/40
3750/3750 [==============================] - 104s 28ms/step - loss: 0.0635 - accuracy: 0.9810 - val_loss: 0.0292 - val_accuracy: 0.9908
Epoch 4/40
3750/3750 [==============================] - 95s 25ms/step - loss: 0.0558 - accuracy: 0.9839 - val_loss: 0.0197 - val_accuracy: 0.9933
Epoch 5/40
3750/3750 [==============================] - 98s 26ms/step - loss: 0.0447 - accuracy: 0.9869 - val_loss: 0.0236 - val_accuracy: 0.9922
Epoch 6/40
3750/3750 [==============================] - 99s 26ms/step - loss: 0.0418 - accuracy: 0.9880 - val_loss: 0.0185 - val_accuracy: 0.9942
Epoch 7/40
1500/3750 [===========>..................] - ETA: 57s - loss: 0.0342 - accuracy: 0.9900